In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas_profiling 

# 전처리
from sklearn.preprocessing import LabelEncoder, StandardScaler

# 데이터 분할
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV

# 모델링
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as lgbm

# 평가지표
from sklearn.metrics import accuracy_score, mean_squared_error
#
import warnings

In [2]:
loan_data = pd.read_csv(r'C:\Users\KYOBO\Desktop\dataset\loan_result.csv')
respec = pd.read_csv(r'C:\Users\KYOBO\Desktop\dataset\respec_final_XGB.csv')

In [3]:
respec

,Unnamed: 0,application_id,user_id,gender,insert_time,credit_score,yearly_income,income_type,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,reage,career
0,0,954900,553686,1.0,2022-06-07 14:29:03,870.0,30000000.0,1,정규직,기타가족소유,30000000.0,SWITCHLOAN,0.0,1.0,1.0,27000000.0,50.0,16.0
1,1,137274,59516,1.0,2022-06-07 21:40:22,710.0,30000000.0,2,기타,기타가족소유,10000000.0,LIVING,1.0,1.0,5.0,15000000.0,20.0,2.0
2,2,1570936,167320,1.0,2022-06-07 09:40:27,820.0,62000000.0,3,정규직,자가,2000000.0,LIVING,1.0,1.0,7.0,344000000.0,30.0,6.0
3,3,967833,33400,1.0,2022-06-07 08:55:07,630.0,36000000.0,3,정규직,기타가족소유,5000000.0,LIVING,0.0,0.0,1.0,16000000.0,20.0,2.0
4,4,1614045,860410,1.0,2022-06-07 11:29:05,590.0,45000000.0,4,기타,자가,15000000.0,SWITCHLOAN,1.0,0.0,1.0,20000000.0,40.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968748,968748,1864587,489900,1.0,2022-03-22 14:55:32,590.0,25000000.0,2,기타,기타가족소유,5000000.0,BUSINESS,0.0,0.0,0.0,0.0,20.0,2.0
968749,968749,1327066,151422,1.0,2022-03-22 01:19:24,980.0,20000000.0,4,기타,자가,50000000.0,LIVING,0.0,1.0,1.0,43500000.0,60.0,6.0
968750,968750,1319606,173524,1.0,2022-03-22 07:34:32,750.0,75000000.0,3,정규직,자가,100000000.0,SWITCHLOAN,1.0,1.0,8.0,200000000.0,40.0,14.0
968751,968751,1482466,766546,1.0,2022-03-22 22:12:35,640.0,50000000.0,3,정규직,자가,10000000.0,SWITCHLOAN,0.0,1.0,10.0,117000000.0,40.0,6.0


respec 로그변환
- yealy income
- desired_amount
- existing_loan_amt

In [4]:
respec['yearly_income']=np.log(respec['yearly_income'])

C:\Users\KYOBO\anaconda3\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [5]:
respec["desired_amount"]=np.log(respec["desired_amount"])

In [6]:
respec["existing_loan_amt"]=np.log(respec["existing_loan_amt"])

범주형 변수들 category형식으로 맞추기

In [7]:
respec['gender'] = pd.Categorical(respec.gender)
respec.dtypes

Unnamed: 0                                int64
application_id                            int64
user_id                                   int64
gender                                 category
insert_time                              object
credit_score                            float64
yearly_income                           float64
income_type                               int64
employment_type                          object
houseown_type                            object
desired_amount                          float64
purpose                                  object
personal_rehabilitation_yn              float64
personal_rehabilitation_complete_yn     float64
existing_loan_cnt                       float64
existing_loan_amt                       float64
reage                                   float64
career                                  float64
dtype: object

In [8]:
respec['personal_rehabilitation_yn'] = pd.Categorical(respec.personal_rehabilitation_yn)
respec.dtypes

Unnamed: 0                                int64
application_id                            int64
user_id                                   int64
gender                                 category
insert_time                              object
credit_score                            float64
yearly_income                           float64
income_type                               int64
employment_type                          object
houseown_type                            object
desired_amount                          float64
purpose                                  object
personal_rehabilitation_yn             category
personal_rehabilitation_complete_yn     float64
existing_loan_cnt                       float64
existing_loan_amt                       float64
reage                                   float64
career                                  float64
dtype: object

In [9]:
respec['personal_rehabilitation_complete_yn'] = pd.Categorical(respec.personal_rehabilitation_complete_yn)
respec.dtypes

Unnamed: 0                                int64
application_id                            int64
user_id                                   int64
gender                                 category
insert_time                              object
credit_score                            float64
yearly_income                           float64
income_type                               int64
employment_type                          object
houseown_type                            object
desired_amount                          float64
purpose                                  object
personal_rehabilitation_yn             category
personal_rehabilitation_complete_yn    category
existing_loan_cnt                       float64
existing_loan_amt                       float64
reage                                   float64
career                                  float64
dtype: object

In [10]:
respec.isnull().sum()

Unnamed: 0                             0
application_id                         0
user_id                                0
gender                                 0
insert_time                            0
credit_score                           0
yearly_income                          0
income_type                            0
employment_type                        0
houseown_type                          0
desired_amount                         0
purpose                                0
personal_rehabilitation_yn             0
personal_rehabilitation_complete_yn    0
existing_loan_cnt                      0
existing_loan_amt                      0
reage                                  0
career                                 0
dtype: int64

loan_data랑 합치고 군집화 진행

In [11]:
loan_data=loan_data.dropna(subset=['loan_rate'])
loan_data

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,NaN
1,1748340,2022-06-07 13:05:41,25,169,24000000.0,17.9,NaN
2,1748340,2022-06-07 13:05:41,2,7,24000000.0,18.5,NaN
3,1748340,2022-06-07 13:05:41,4,268,29000000.0,10.8,NaN
4,1748340,2022-06-07 13:05:41,11,118,5000000.0,16.4,NaN
...,...,...,...,...,...,...,...
13527358,1428218,2022-06-03 12:01:58,62,200,3000000.0,14.8,NaN
13527359,1428218,2022-06-03 12:01:49,2,7,40000000.0,11.8,NaN
13527360,1428218,2022-06-03 12:01:48,32,257,15000000.0,7.2,NaN
13527361,1428218,2022-06-03 12:01:48,33,110,44000000.0,13.5,NaN


In [12]:
loan_train=loan_data[(loan_data['is_applied']==0)|(loan_data['is_applied']==1)]
loan_train

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
13284,2157865,2022-05-09 08:44:59,54,235,20000000.0,16.5,1.0
13285,576643,2022-05-09 10:54:53,54,235,11000000.0,16.5,0.0
13286,576643,2022-05-09 10:54:53,11,118,3000000.0,20.0,0.0
13287,2136706,2022-05-09 10:41:06,42,216,10000000.0,13.5,0.0
13288,2136706,2022-05-09 10:41:07,25,169,22000000.0,15.9,0.0
...,...,...,...,...,...,...,...
13519634,1969227,2022-05-16 14:42:58,2,7,30000000.0,13.6,0.0
13519635,1969227,2022-05-16 14:42:57,33,110,9000000.0,14.4,0.0
13519636,1969227,2022-05-16 14:42:56,50,142,3000000.0,11.2,0.0
13519637,1969227,2022-05-16 14:43:18,22,100,4000000.0,15.3,0.0


In [13]:
loan_target=loan_data[loan_data.is_applied.isnull()]
loan_target

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,NaN
1,1748340,2022-06-07 13:05:41,25,169,24000000.0,17.9,NaN
2,1748340,2022-06-07 13:05:41,2,7,24000000.0,18.5,NaN
3,1748340,2022-06-07 13:05:41,4,268,29000000.0,10.8,NaN
4,1748340,2022-06-07 13:05:41,11,118,5000000.0,16.4,NaN
...,...,...,...,...,...,...,...
13527358,1428218,2022-06-03 12:01:58,62,200,3000000.0,14.8,NaN
13527359,1428218,2022-06-03 12:01:49,2,7,40000000.0,11.8,NaN
13527360,1428218,2022-06-03 12:01:48,32,257,15000000.0,7.2,NaN
13527361,1428218,2022-06-03 12:01:48,33,110,44000000.0,13.5,NaN


In [14]:
df = pd.merge(loan_train, respec,
                 how = 'left',
                 on = 'application_id')
df

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,Unnamed: 0,user_id,gender,...,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,reage,career
0,2157865,2022-05-09 08:44:59,54,235,20000000.0,16.5,1.0,471355,346970,1.0,...,정규직,자가,16.118096,LIVING,1.0,0.0,12.0,18.903107,50.0,10.0
1,576643,2022-05-09 10:54:53,54,235,11000000.0,16.5,0.0,284656,545882,1.0,...,정규직,전월세,16.811243,LIVING,1.0,0.0,7.0,18.478950,40.0,4.0
2,576643,2022-05-09 10:54:53,11,118,3000000.0,20.0,0.0,284656,545882,1.0,...,정규직,전월세,16.811243,LIVING,1.0,0.0,7.0,18.478950,40.0,4.0
3,2136706,2022-05-09 10:41:06,42,216,10000000.0,13.5,0.0,441982,558819,1.0,...,정규직,전월세,18.197537,SWITCHLOAN,0.0,1.0,2.0,17.875954,40.0,11.0
4,2136706,2022-05-09 10:41:07,25,169,22000000.0,15.9,0.0,441982,558819,1.0,...,정규직,전월세,18.197537,SWITCHLOAN,0.0,1.0,2.0,17.875954,40.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10264381,1969227,2022-05-16 14:42:58,2,7,30000000.0,13.6,0.0,400358,109899,1.0,...,정규직,자가,16.811243,SWITCHLOAN,1.0,1.0,9.0,18.488339,40.0,2.0
10264382,1969227,2022-05-16 14:42:57,33,110,9000000.0,14.4,0.0,400358,109899,1.0,...,정규직,자가,16.811243,SWITCHLOAN,1.0,1.0,9.0,18.488339,40.0,2.0
10264383,1969227,2022-05-16 14:42:56,50,142,3000000.0,11.2,0.0,400358,109899,1.0,...,정규직,자가,16.811243,SWITCHLOAN,1.0,1.0,9.0,18.488339,40.0,2.0
10264384,1969227,2022-05-16 14:43:18,22,100,4000000.0,15.3,0.0,400358,109899,1.0,...,정규직,자가,16.811243,SWITCHLOAN,1.0,1.0,9.0,18.488339,40.0,2.0


In [15]:
from kmodes.kprototypes import KPrototypes
import plotly.express as px

In [16]:
df = df[['application_id','bank_id','product_id','loan_limit','loan_rate','is_applied','credit_score']]

In [17]:
df

,application_id,bank_id,product_id,loan_limit,loan_rate,is_applied,credit_score
0,2157865,54,235,20000000.0,16.5,1.0,540.0
1,576643,54,235,11000000.0,16.5,0.0,580.0
2,576643,11,118,3000000.0,20.0,0.0,580.0
3,2136706,42,216,10000000.0,13.5,0.0,740.0
4,2136706,25,169,22000000.0,15.9,0.0,740.0
...,...,...,...,...,...,...,...
10264381,1969227,2,7,30000000.0,13.6,0.0,600.0
10264382,1969227,33,110,9000000.0,14.4,0.0,600.0
10264383,1969227,50,142,3000000.0,11.2,0.0,600.0
10264384,1969227,22,100,4000000.0,15.3,0.0,600.0


In [19]:
normalized_df=(df['loan_limit']-df['loan_limit'].mean())/df['loan_limit'].std()

In [20]:
normalized_df

0          -0.116924
1          -0.520378
2          -0.879003
3          -0.565206
4          -0.027268
              ...   
10264381    0.331357
10264382   -0.610034
10264383   -0.879003
10264384   -0.834175
10264385   -0.610034
Name: loan_limit, Length: 10264386, dtype: float64

In [21]:
df['loan_limit'] = normalized_df

C:\Users\KYOBO\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
df

,application_id,bank_id,product_id,loan_limit,loan_rate,is_applied,credit_score
0,2157865,54,235,-0.116924,16.5,1.0,540.0
1,576643,54,235,-0.520378,16.5,0.0,580.0
2,576643,11,118,-0.879003,20.0,0.0,580.0
3,2136706,42,216,-0.565206,13.5,0.0,740.0
4,2136706,25,169,-0.027268,15.9,0.0,740.0
...,...,...,...,...,...,...,...
10264381,1969227,2,7,0.331357,13.6,0.0,600.0
10264382,1969227,33,110,-0.610034,14.4,0.0,600.0
10264383,1969227,50,142,-0.879003,11.2,0.0,600.0
10264384,1969227,22,100,-0.834175,15.3,0.0,600.0


In [ ]:
kp = KPrototypes(n_clusters=4, init='Huang', n_init=1, verbose=True)
kp.fit_predict(df, categorical=[4])

In [ ]:
print(kp.labels_)

In [ ]:
df['is_applied'] = df.is_applied
df['cluster']=kp.labels_
df_result = df.groupby(['is_applied','cluster'])['product_id'].count()
print(df_result)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_transformed = pca.fit_transform(df)

df['pca_x'] = pca_transformed[:,0]
df['pca_y'] = pca_transformed[:,1]
df.head(3)

In [ ]:
# cluster 값이 0, 1, 2 인 경우마다 별도의 Index로 추출
marker0_ind = df[df['cluster']==0].index
marker1_ind = df[df['cluster']==1].index
marker2_ind = df[df['cluster']==2].index

# cluster값 0, 1, 2에 해당하는 Index로 각 cluster 레벨의 pca_x, pca_y 값 추출. o, s, ^ 로 marker 표시
plt.scatter(x=df.loc[marker0_ind,'pca_x'], y=df.loc[marker0_ind,'pca_y'], marker='o') 
plt.scatter(x=df.loc[marker1_ind,'pca_x'], y=df.loc[marker1_ind,'pca_y'], marker='s')
plt.scatter(x=df.loc[marker2_ind,'pca_x'], y=df.loc[marker2_ind,'pca_y'], marker='^')

plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('3 Clusters Visualization by 2 PCA Components')
plt.savefig('test.png') 
plt.show()

In [ ]:
df.head()